In [11]:
from sklearn import datasets
import pandas as pd
pd.set_option("display.max_columns", 100)
from model_comparison import ModelComparison, TaskName, ModelName


In [12]:
comparison_datasets = {}

# California housing

In [13]:
california_housing = datasets.fetch_california_housing(as_frame=True)
california_housing.data.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [14]:
california_housing.data.shape

(20640, 8)

In [15]:
california_housing.target.head()

0    4.526
1    3.585
2    3.521
3    3.413
4    3.422
Name: MedHouseVal, dtype: float64

In [17]:
comparison_datasets['california'] = {"task": TaskName.regression, 
                                     "features": california_housing.data, 
                                     "target": california_housing.target,
                                     "cv": 4}

# Covtype

In [18]:
covtype_features, covtype_target = datasets.fetch_covtype(return_X_y=True)

In [19]:
covtype_features

array([[2.596e+03, 5.100e+01, 3.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [2.590e+03, 5.600e+01, 2.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [2.804e+03, 1.390e+02, 9.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       ...,
       [2.386e+03, 1.590e+02, 1.700e+01, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [2.384e+03, 1.700e+02, 1.500e+01, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [2.383e+03, 1.650e+02, 1.300e+01, ..., 0.000e+00, 0.000e+00,
        0.000e+00]])

In [20]:
covtype_features.shape

(581012, 54)

In [21]:
covtype_target

array([5, 5, 2, ..., 3, 3, 3], dtype=int32)

In [22]:
comparison_datasets['covtype'] = {"task": TaskName.classification, 
                                     "features": pd.DataFrame(covtype_features), 
                                     "target": pd.Series(covtype_target),
                                     "cv": 2}

# Adult

In [23]:
adult = datasets.fetch_openml('adult', as_frame=True)

/Users/victorlandeau/Library/Caches/pypoetry/virtualenvs/tabular-models-benchmark-BMhtYu43-py3.8/lib/python3.8/site-packages/sklearn/datasets/_openml.py:373: UserWarning: Multiple active versions of the dataset matching the name adult exist. Versions may be fundamentally different, returning version 1.
  warn("Multiple active versions of the dataset matching the name"


In [24]:
adult.data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native-country
0,2,State-gov,77516.0,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,1,0,2,United-States
1,3,Self-emp-not-inc,83311.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,0,United-States
2,2,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,2,United-States
3,3,Private,234721.0,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,2,United-States
4,1,Private,338409.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,2,Cuba


In [25]:
adult.data.shape

(48842, 14)

In [26]:
adult.target.head()

0    <=50K
1    <=50K
2    <=50K
3    <=50K
4    <=50K
Name: class, dtype: category
Categories (2, object): ['>50K', '<=50K']

In [27]:
comparison_datasets['adult'] = {"task": TaskName.classification, 
                                "features": adult.data, 
                                "target": adult.target,
                                "cv": 4}


# UKAir

In [28]:
ukair = datasets.fetch_openml('particulate-matter-ukair-2017', as_frame=True)

In [29]:
ukair.data.head()

,datetime,Hour,Month,DayofWeek,Site.Name,Environment.Type,Zone,Altitude..m.,PM.sub.2.5..sub..particulate.matter..Hourly.measured.
0,2017-01-01 01:00:00,1.0,1,1,Aberdeen,Background Urban,North East Scotland,20.0,9.9
1,2017-01-01 02:00:00,2.0,1,1,Aberdeen,Background Urban,North East Scotland,20.0,4.6
2,2017-01-01 03:00:00,3.0,1,1,Aberdeen,Background Urban,North East Scotland,20.0,0.8
3,2017-01-01 04:00:00,4.0,1,1,Aberdeen,Background Urban,North East Scotland,20.0,2.1
4,2017-01-01 05:00:00,5.0,1,1,Aberdeen,Background Urban,North East Scotland,20.0,5.4


In [30]:
ukair.data.shape

(394299, 9)

In [31]:
ukair.target.head()

0    12.8
1    11.0
2     4.5
3     7.3
4     8.2
Name: PM.sub.10..sub..particulate.matter..Hourly.measured., dtype: float64

In [33]:
comparison_datasets['ukair'] = {"task": TaskName.regression, 
                                "features": ukair.data, 
                                "target": ukair.target,
                                "cv": 2}

# Diabetes

In [34]:
diabetes = datasets.fetch_openml('diabetes', as_frame=True)

/Users/victorlandeau/Library/Caches/pypoetry/virtualenvs/tabular-models-benchmark-BMhtYu43-py3.8/lib/python3.8/site-packages/sklearn/datasets/_openml.py:373: UserWarning: Multiple active versions of the dataset matching the name diabetes exist. Versions may be fundamentally different, returning version 1.
  warn("Multiple active versions of the dataset matching the name"


In [35]:
diabetes.data.head()

,preg,plas,pres,skin,insu,mass,pedi,age
0,6.0,148.0,72.0,35.0,0.0,33.6,0.627,50.0
1,1.0,85.0,66.0,29.0,0.0,26.6,0.351,31.0
2,8.0,183.0,64.0,0.0,0.0,23.3,0.672,32.0
3,1.0,89.0,66.0,23.0,94.0,28.1,0.167,21.0
4,0.0,137.0,40.0,35.0,168.0,43.1,2.288,33.0


In [36]:
diabetes.data.shape

(768, 8)

In [37]:
diabetes.target.head()

0    tested_positive
1    tested_negative
2    tested_positive
3    tested_negative
4    tested_positive
Name: class, dtype: category
Categories (2, object): ['tested_negative', 'tested_positive']

In [38]:
comparison_datasets['diabetes'] = {"task": TaskName.classification, 
                                   "features": diabetes.data, 
                                   "target": diabetes.target,
                                   "cv": 10}

# Bank marketing

In [39]:
bank_marketing = datasets.fetch_openml("bank-marketing", as_frame=True)

/Users/victorlandeau/Library/Caches/pypoetry/virtualenvs/tabular-models-benchmark-BMhtYu43-py3.8/lib/python3.8/site-packages/sklearn/datasets/_openml.py:373: UserWarning: Multiple active versions of the dataset matching the name bank-marketing exist. Versions may be fundamentally different, returning version 1.
  warn("Multiple active versions of the dataset matching the name"


In [40]:
bank_marketing.data.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16
0,58.0,management,married,tertiary,no,2143.0,yes,no,unknown,5.0,may,261.0,1.0,-1.0,0.0,unknown
1,44.0,technician,single,secondary,no,29.0,yes,no,unknown,5.0,may,151.0,1.0,-1.0,0.0,unknown
2,33.0,entrepreneur,married,secondary,no,2.0,yes,yes,unknown,5.0,may,76.0,1.0,-1.0,0.0,unknown
3,47.0,blue-collar,married,unknown,no,1506.0,yes,no,unknown,5.0,may,92.0,1.0,-1.0,0.0,unknown
4,33.0,unknown,single,unknown,no,1.0,no,no,unknown,5.0,may,198.0,1.0,-1.0,0.0,unknown


In [41]:
bank_marketing.data.shape

(45211, 16)

In [42]:
bank_marketing.target.head()

0    1
1    1
2    1
3    1
4    1
Name: Class, dtype: category
Categories (2, object): ['1', '2']

In [43]:
comparison_datasets['bank'] = {"task": TaskName.classification, 
                               "features": bank_marketing.data, 
                               "target": bank_marketing.target,
                               "cv": 4}

# Speed dating

In [44]:
speed_dating = datasets.fetch_openml("SpeedDating", as_frame=True)

In [45]:
speed_dating.data.head()

,has_null,wave,gender,age,age_o,d_age,d_d_age,race,race_o,samerace,importance_same_race,importance_same_religion,d_importance_same_race,d_importance_same_religion,field,pref_o_attractive,pref_o_sincere,pref_o_intelligence,pref_o_funny,pref_o_ambitious,pref_o_shared_interests,d_pref_o_attractive,d_pref_o_sincere,d_pref_o_intelligence,d_pref_o_funny,d_pref_o_ambitious,d_pref_o_shared_interests,attractive_o,sinsere_o,intelligence_o,funny_o,ambitous_o,shared_interests_o,d_attractive_o,d_sinsere_o,d_intelligence_o,d_funny_o,d_ambitous_o,d_shared_interests_o,attractive_important,sincere_important,intellicence_important,funny_important,ambtition_important,shared_interests_important,d_attractive_important,d_sincere_important,d_intellicence_important,d_funny_important,d_ambtition_important,...,d_funny_partner,d_ambition_partner,d_shared_interests_partner,sports,tvsports,exercise,dining,museums,art,hiking,gaming,clubbing,reading,tv,theater,movies,concerts,music,shopping,yoga,d_sports,d_tvsports,d_exercise,d_dining,d_museums,d_art,d_hiking,d_gaming,d_clubbing,d_reading,d_tv,d_theater,d_movies,d_concerts,d_music,d_shopping,d_yoga,interests_correlate,d_interests_correlate,expected_happy_with_sd_people,expected_num_interested_in_me,expected_num_matches,d_expected_happy_with_sd_people,d_expected_num_interested_in_me,d_expected_num_matches,like,guess_prob_liked,d_like,d_guess_prob_liked,met
0,0,1.0,female,21.0,27.0,6.0,[4-6],Asian/Pacific Islander/Asian-American,European/Caucasian-American,0,2.0,4.0,[2-5],[2-5],Law,35.0,20.0,20.0,20.0,0.0,5.0,[21-100],[16-20],[16-20],[16-20],[0-15],[0-15],6.0,8.0,8.0,8.0,8.0,6.0,[6-8],[6-8],[6-8],[6-8],[6-8],[6-8],15.0,20.0,20.0,15.0,15.0,15.0,[0-15],[16-20],[16-20],[0-15],[0-15],...,[6-8],[6-8],[0-5],9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,[9-10],[0-5],[6-8],[9-10],[0-5],[0-5],[0-5],[0-5],[0-5],[6-8],[9-10],[0-5],[9-10],[9-10],[9-10],[6-8],[0-5],0.14,[0-0.33],3.0,2.0,4.0,[0-4],[0-3],[3-5],7.0,6.0,[6-8],[5-6],0.0
1,0,1.0,female,21.0,22.0,1.0,[0-1],Asian/Pacific Islander/Asian-American,European/Caucasian-American,0,2.0,4.0,[2-5],[2-5],Law,60.0,0.0,0.0,40.0,0.0,0.0,[21-100],[0-15],[0-15],[21-100],[0-15],[0-15],7.0,8.0,10.0,7.0,7.0,5.0,[6-8],[6-8],[9-10],[6-8],[6-8],[0-5],15.0,20.0,20.0,15.0,15.0,15.0,[0-15],[16-20],[16-20],[0-15],[0-15],...,[6-8],[0-5],[6-8],9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,[9-10],[0-5],[6-8],[9-10],[0-5],[0-5],[0-5],[0-5],[0-5],[6-8],[9-10],[0-5],[9-10],[9-10],[9-10],[6-8],[0-5],0.54,[0.33-1],3.0,2.0,4.0,[0-4],[0-3],[3-5],7.0,5.0,[6-8],[5-6],1.0
2,1,1.0,female,21.0,22.0,1.0,[0-1],Asian/Pacific Islander/Asian-American,Asian/Pacific Islander/Asian-American,1,2.0,4.0,[2-5],[2-5],Law,19.0,18.0,19.0,18.0,14.0,12.0,[16-20],[16-20],[16-20],[16-20],[0-15],[0-15],10.0,10.0,10.0,10.0,10.0,10.0,[9-10],[9-10],[9-10],[9-10],[9-10],[9-10],15.0,20.0,20.0,15.0,15.0,15.0,[0-15],[16-20],[16-20],[0-15],[0-15],...,[6-8],[0-5],[6-8],9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,[9-10],[0-5],[6-8],[9-10],[0-5],[0-5],[0-5],[0-5],[0-5],[6-8],[9-10],[0-5],[9-10],[9-10],[9-10],[6-8],[0-5],0.16,[0-0.33],3.0,2.0,4.0,[0-4],[0-3],[3-5],7.0,NaN,[6-8],[0-4],1.0
3,0,1.0,female,21.0,23.0,2.0,[2-3],Asian/Pacific Islander/Asian-American,European/Caucasian-American,0,2.0,4.0,[2-5],[2-5],Law,30.0,5.0,15.0,40.0,5.0,5.0,[21-100],[0-15],[0-15],[21-100],[0-15],[0-15],7.0,8.0,9.0,8.0,9.0,8.0,[6-8],[6-8],[9-10],[6-8],[9-10],[6-8],15.0,20.0,20.0,15.0,15.0,15.0,[0-15],[16-20],[16-20],[0-15],[0-15],...,[6-8],[6-8],[6-8],9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,[9-10],[0-5],[6-8],[9-10],[0-5],[0-5],[0-5],[0-5],[0-5],[6-8],[9-10],[0-5],[9-10],[9-10],[9-10],[6-8],[0-5],0.61,[0.33-1],3.0,2.0,4.0,[0-4],[0-3],[3-5],7.0,6.0,[6-8],[5-6],0.0
4,0,1.0,female,21.0,24.0,3.0,[2-3],Asian/Pacific Islander/Asian-American,Latino/Hispanic American,0,2.0,4.0,[2-5],[2-5],Law,30.0,10.0,20.0,10.0,10.0,20.0,[21-100],[0-15],[16-20],[0-15],[0-15],

In [46]:
speed_dating.data.shape

(8378, 120)

In [47]:
speed_dating.target.head()

0    0
1    0
2    1
3    1
4    1
Name: match, dtype: category
Categories (2, object): ['0', '1']

In [48]:
comparison_datasets['dating'] = {"task": TaskName.classification, 
                                 "features": speed_dating.data, 
                                 "target": speed_dating.target,
                                 "cv": 6}

# Hill valley

In [49]:
hill_valley = datasets.fetch_openml("hill-valley", as_frame=True)

/Users/victorlandeau/Library/Caches/pypoetry/virtualenvs/tabular-models-benchmark-BMhtYu43-py3.8/lib/python3.8/site-packages/sklearn/datasets/_openml.py:373: UserWarning: Multiple active versions of the dataset matching the name hill-valley exist. Versions may be fundamentally different, returning version 1.
  warn("Multiple active versions of the dataset matching the name"


In [50]:
hill_valley.data.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100
0,39.02,36.49,38.20,38.85,39.38,39.74,37.02,39.53,38.81,38.79,37.65,39.34,38.55,39.03,37.21,36.32,37.81,38.95,36.70,39.72,37.06,37.29,36.43,36.53,36.19,38.17,37.30,36.15,36.68,36.70,36.68,36.99,38.92,37.25,37.47,36.32,35.75,35.68,34.66,34.26,35.62,36.60,34.78,34.67,34.30,33.40,31.40,31.75,31.75,32.84,33.76,35.74,34.01,33.91,36.88,34.41,35.52,36.94,36.95,35.57,38.02,37.32,39.05,37.97,37.01,38.98,38.83,38.87,38.03,38.40,38.25,38.61,36.23,37.81,37.98,38.58,38.96,38.97,39.08,38.79,38.79,36.31,36.59,38.19,37.95,39.63,39.27,37.19,37.13,37.47,37.57,36.62,36.92,38.80,38.52,38.07,36.73,39.46,37.50,39.10
1,1.83,1.71,1.77,1.77,1.68,1.78,1.80,1.70,1.75,1.78,1.86,1.76,1.81,1.86,1.74,1.78,1.81,2.02,2.00,2.01,2.00,2.06,2.00,1.93,1.88,1.85,1.89,1.83,1.76,1.83,1.81,1.81,1.78,1.85,1.86,1.73,1.79,1.81,1.85,1.71,1.71,1.71,1.84,1.76,1.73,1.83,1.68,1.73,1.76,1.77,1.72,1.75,1.66,1.76,1.77,1.78,1.63,1.72,1.66,1.67,1.74,1.65,1.74,1.79,1.69,1.76,1.74,1.82,1.78,1.65,1.65,1.82,1.71,1.83,1.72,1.63,1.77,1.69,1.81,1.74,1.70,1.72,1.74,1.72,1.74,1.71,1.70,1.83,1.79,1.78,1.71,1.80,1.79,1.77,1.74,1.74,1.80,1.78,1.75,1.69
2,68177.69,66138.42,72981.88,74304.33,67549.66,69367.34,69169.41,73268.61,74465.84,72503.37,73417.66,67759.27,74608.57,66336.20,75236.92,70754.31,67700.92,73902.96,66161.93,71646.83,72194.27,69116.92,71612.47,74770.43,69686.94,74502.98,71904.31,70358.08,69694.61,73941.16,75446.14,75816.84,73435.10,66432.13,70823.56,68920.89,72429.82,69138.09,74919.85,75314.11,72400.71,74763.70,67268.51,67508.12,71560.35,71603.35,76256.25,72295.62,74468.56,75270.67,68124.82,70584.36,69655.63,67212.35,70308.43,66559.58,69940.83,69125.90,69415.46,69805.46,70860.60,74298.25,69924.44,75101.58,69584.85,70171.32,74564.11,75450.36,70029.97,69964.37,73687.75,72576.15,73573.75,76410.68,83257.85,76957.31,80125.79,83178.08,86976.04,91197.47,80400.37,82945.32,80513.74,78105.50,79752.76,70485.77,74493.52,74222.23,75592.31,72895.84,69384.71,73438.88,71053.35,71112.62,74916.48,72571.58,66348.97,71063.72,67404.27,74920.24
3,44889.06,39191.86,40728.46,38576.36,45876.06,47034.00,46611.43,37668.32,40980.89,38466.15,39113.55,43060.28,33587.90,32416.01,33536.08,30160.05,34261.28,30575.53,28895.52,27563.45,26946.39,35360.14,34731.21,32547.20,37601.73,43350.41,43830.86,37540.38,42492.45,41949.33,40540.92,39004.88,41387.14,38493.69,39862.56,42408.36,39649.30,41907.03,48439.05,46254.42,44949.16,40162.05,44500.79,39445.87,44434.49,46671.39,46820.18,40639.60,42577.66,42217.47,44160.92,46520.63,41933.41,40986.25,44426.07,41605.84,48896.86,44774.73,42449.71,39905.29,39907.35,45465.54,40921.92,41465.08,46174.22,41163.16,44553.21,48607.01,40804.21,45359.30,45375.30,43324.00,40089.31,43590.66,44508.23,44542.14,40139.58,43882.35,44777.21,41549.98,47769.58,46012.65,41979.51,40700.71,48893.58,40028.35,46377.96,48162.67,47866.58,39834.12,47653.60,42625.67,40684.20,46960.73,44546.80,45410.53,47139.44,43095.68,40888.34,39615.19
4,5.70,5.40,5.28,5.38,5.27,5.61,6.00,5.38,5.34,5.87,5.54,5.60,5.71,5.67,5.68,5.37,5.37,5.33,5.49,5.33,5.37,5.33,5.68,5.96,5.34,5.94,5.30,5.80,5.47,5.58,5.37,5.57,5.60,5.90,5.21,5.19,5.63,5.58,5.26,5.32,5.17,5.20,5.57,5.85,5.22,5.34,6.03,5.99,5.90,5.69,5.35,5.74,5.99,5.71,5.77,5.49,5.69,5.48,5.50,5.21,5.72,5.44,5.21,5.70,5.92,5.47,5.52,5.19,5.11,5.65,5.12,5.07,4.89,4.90,4.70,4.90,3.96,3.61,4.62,4.35,4.44,4.97,5.33,5.12,5.51,4.93,5.34,5.07,5.29,5.82,5.52,5.17,5.67,5.60,5.94,5.73,5.22,5.30,5.73,5.91


In [51]:
hill_valley.data.shape

(1212, 100)

In [52]:
hill_valley.target.head()

0    0
1    1
2    1
3    0
4    0
Name: Class, dtype: category
Categories (2, object): ['0', '1']

In [53]:
comparison_datasets['valley'] = {"task": TaskName.classification, 
                                 "features": hill_valley.data, 
                                 "target": hill_valley.target,
                                 "cv": 8}

# Cars

In [54]:
cars = pd.read_csv("cars.csv")

In [55]:
cars.head()

,manufacturer_name,model_name,transmission,color,odometer_value,year_produced,engine_fuel,engine_has_gas,engine_type,engine_capacity,body_type,has_warranty,state,drivetrain,price_usd,is_exchangeable,location_region,number_of_photos,up_counter,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,duration_listed
0,Subaru,Outback,automatic,silver,190000,2010,gasoline,False,gasoline,2.5,universal,False,owned,all,10900.00,False,Минская обл.,9,13,False,True,True,True,False,True,False,True,True,True,16
1,Subaru,Outback,automatic,blue,290000,2002,gasoline,False,gasoline,3.0,universal,False,owned,all,5000.00,True,Минская обл.,12,54,False,True,False,False,True,True,False,False,False,True,83
2,Subaru,Forester,automatic,red,402000,2001,gasoline,False,gasoline,2.5,suv,False,owned,all,2800.00,True,Минская обл.,4,72,False,True,False,False,False,False,False,False,True,True,151
3,Subaru,Impreza,mechanical,blue,10000,1999,gasoline,False,gasoline,3.0,sedan,False,owned,all,9999.00,True,Минская обл.,9,42,True,False,False,False,False,False,False,False,False,False,86
4,Subaru,Legacy,automatic,black,280000,2001,gasoline,False,gasoline,2.5,universal,False,owned,all,2134.11,True,Гомельская обл.,14,7,False,True,False,True,True,False,False,False,False,True,7


In [56]:
cars.shape

(38531, 30)

In [57]:
cars_target = "duration_listed"
comparison_datasets['cars'] = {"task": TaskName.regression, 
                                 "features": cars.drop(columns=cars_target), 
                                 "target": cars[cars_target],
                                 "cv": 4}

# Comparaison

In [46]:
def get_comparison(dataset_infos):
    comparison = ModelComparison(task_name=dataset_infos["task"], 
                             cross_validation_n_folds=dataset_infos["cv"], 
                             features=dataset_infos["features"], 
                             target=dataset_infos["target"])
    return comparison.get_default_models_scores_and_training_time()

In [47]:
perf_comparisons = {dataset_name: get_comparison(comparison_datasets[dataset_name])
                   for dataset_name in comparison_datasets.keys()}
perf_comparisons

/Users/victorlandeau/Library/Caches/pypoetry/virtualenvs/tabular-models-benchmark-BMhtYu43-py3.8/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


{'california': {<ModelNames.catboost: 'catboost'>: (0.8503926820632256,
   23.941099882125854),
  <ModelNames.lightgbm: 'lightgbm'>: (0.8364988580304382, 1.0286211967468262),
  <ModelNames.lightgbm_with_cat_encoder: 'lightgbm_with_cat_encoder'>: (0.8363076909236052,
   1.1544129848480225),
  <ModelNames.xgboost: 'xgboost'>: (0.8343045611041091, 2.5886080265045166),
  <ModelNames.xgboost_with_cat_encoder: 'xgboost_with_cat_encoder'>: (0.8318651968168912,
   3.068925142288208),
  <ModelNames.lightgbm_with_features_generator: 'lightgbm_with_features_generator'>: (0.8362838196020874,
   4.576110124588013)},
 'covtype': {<ModelNames.catboost: 'catboost'>: (0.8813019352440226,
   126.73606896400452),
  <ModelNames.lightgbm: 'lightgbm'>: (0.8521097670960325, 24.178813934326172),
  <ModelNames.lightgbm_with_cat_encoder: 'lightgbm_with_cat_encoder'>: (0.8524298981776625,
   23.447602033615112),
  <ModelNames.xgboost: 'xgboost'>: (0.8693073464919829, 136.78466320037842),
  <ModelNames.xgboost_wi

In [52]:
lightgbm_performance = [perf_comparisons[dataset_perfs][ModelNames.lightgbm][0]
                       for dataset_perfs in perf_comparisons.keys()]
catboost_performance = [perf_comparisons[dataset_perfs][ModelNames.catboost][0]
                       for dataset_perfs in perf_comparisons.keys()]
xgboost_performance = [perf_comparisons[dataset_perfs][ModelNames.xgboost][0]
                       for dataset_perfs in perf_comparisons.keys()]
xgboost_cat_performance = [perf_comparisons[dataset_perfs][ModelNames.xgboost_with_cat_encoder][0]
                       for dataset_perfs in perf_comparisons.keys()]
lightgbm_cat_performance = [perf_comparisons[dataset_perfs][ModelNames.lightgbm_with_cat_encoder][0]
                       for dataset_perfs in perf_comparisons.keys()]
lightgbm_feat_gen_performance = [perf_comparisons[dataset_perfs][ModelNames.lightgbm_with_features_generator][0]
                       for dataset_perfs in perf_comparisons.keys()]

/Users/victorlandeau/Library/Caches/pypoetry/virtualenvs/tabular-models-benchmark-BMhtYu43-py3.8/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [53]:
perfs_per_dataset = pd.DataFrame({"dataset": perf_comparisons.keys(),
                                 "lightgbm": [l / x for l, x in zip(lightgbm_performance, xgboost_performance)],
                                 "lightgbm_cat": [lc / x for lc, x in zip(lightgbm_cat_performance, xgboost_performance)],
                                 "lightgbm_feat_gen": [lc / x for lc, x in zip(lightgbm_feat_gen_performance, xgboost_performance)],
                                 "catboost": [c / x for c, x in zip(catboost_performance, xgboost_performance)],
                                 "xgboost_cat": [xc / x for xc, x in zip(xgboost_cat_performance, xgboost_performance)],
                                 "dataset_length": [len(dataset["features"])
                                                   for dataset in comparison_datasets.values()]})
perfs_per_dataset.sort_values("dataset_length")

/Users/victorlandeau/Library/Caches/pypoetry/virtualenvs/tabular-models-benchmark-BMhtYu43-py3.8/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



,dataset,lightgbm,lightgbm_cat,lightgbm_feat_gen,catboost,xgboost_cat,dataset_length
4,diabetes,1.021375,1.031818,1.024848,1.049464,1.017925,768
7,valley,1.084315,1.051218,1.046688,0.982138,1.024256,1212
6,dating,1.000685,1.002473,1.004539,0.999860,0.984188,8378
0,california,1.002630,1.002401,1.002372,1.019283,0.997076,20640
8,cars,1.073726,1.077800,1.086018,1.094558,0.271504,38531
5,bank,1.004424,1.004571,1.004107,1.006062,0.785554,45211
2,adult,1.003544,1.004335,1.005125,1.006658,0.998084,48842
3,ukair,0.967228,0.966036,0.967084,0.978747,0.903791,394299
1,covtype,0.980217,0.980585,0.976382,1.013798,0.998238,581012


In [54]:
import plotly.express as px
px.scatter(perfs_per_dataset, x="dataset_length", y="lightgbm", log_x=True)


/Users/victorlandeau/Library/Caches/pypoetry/virtualenvs/tabular-models-benchmark-BMhtYu43-py3.8/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [57]:
import plotly.express as px
px.scatter(perfs_per_dataset, x="dataset_length", y="catboost", log_x=True)


/Users/victorlandeau/Library/Caches/pypoetry/virtualenvs/tabular-models-benchmark-BMhtYu43-py3.8/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [55]:
import plotly.graph_objects as go

fig = go.Figure(data=[
    go.Bar(name="catboost", x=list(perf_comparisons.keys()), y=catboost_performance),
    go.Bar(name="lightgbm", x=list(perf_comparisons.keys()), y=lightgbm_performance),
    go.Bar(name="xgboost", x=list(perf_comparisons.keys()), y=xgboost_performance),
    go.Bar(name="xgboost_cat", x=list(perf_comparisons.keys()), y=xgboost_cat_performance),
    go.Bar(name="lightgbm_cat", x=list(perf_comparisons.keys()), y=lightgbm_cat_performance),
    go.Bar(name="lightgbm_feat_gen", x=list(perf_comparisons.keys()), y=lightgbm_feat_gen_performance),
           ])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

/Users/victorlandeau/Library/Caches/pypoetry/virtualenvs/tabular-models-benchmark-BMhtYu43-py3.8/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



# Comparaison of tuned models

In [ ]:
from tuned_model_comparison import TunedModelComparison
dataset_infos = comparison_datasets["cars"]
comparison = TunedModelComparison(task_name=dataset_infos["task"], 
                         cross_validation_n_folds=dataset_infos["cv"], 
                         features=dataset_infos["features"], 
                         target=dataset_infos["target"])
cars_tuned_comparison = comparison.get_default_models_scores_and_training_time()
cars_tuned_comparison

[I 2021-02-02 17:01:03,952] A new study created in memory with name: no-name-aee14406-5774-4bd7-9886-3a7e3a05584c


0:	learn: 104.9499167	total: 64.2ms	remaining: 1m 4s
1:	learn: 98.5239928	total: 86.3ms	remaining: 43.1s
2:	learn: 93.3889598	total: 108ms	remaining: 36s
3:	learn: 89.5337988	total: 131ms	remaining: 32.7s
4:	learn: 86.3046730	total: 154ms	remaining: 30.7s
5:	learn: 83.7596654	total: 180ms	remaining: 29.8s
6:	learn: 81.7912416	total: 202ms	remaining: 28.6s
7:	learn: 80.3516620	total: 224ms	remaining: 27.7s
8:	learn: 79.2060965	total: 245ms	remaining: 27s
9:	learn: 78.1359473	total: 267ms	remaining: 26.5s
10:	learn: 77.3714216	total: 290ms	remaining: 26.1s
11:	learn: 76.6098380	total: 313ms	remaining: 25.8s
12:	learn: 76.0886050	total: 335ms	remaining: 25.4s
13:	learn: 75.7328617	total: 349ms	remaining: 24.6s
14:	learn: 75.0895716	total: 371ms	remaining: 24.4s
15:	learn: 74.4975574	total: 398ms	remaining: 24.5s
16:	learn: 74.1436493	total: 422ms	remaining: 24.4s
17:	learn: 74.0138031	total: 433ms	remaining: 23.6s
18:	learn: 73.8674287	total: 449ms	remaining: 23.2s
19:	learn: 73.6176080	t

165:	learn: 61.0983054	total: 3.86s	remaining: 19.4s
166:	learn: 61.0713624	total: 3.89s	remaining: 19.4s
167:	learn: 61.0553849	total: 3.91s	remaining: 19.4s
168:	learn: 61.0292012	total: 3.94s	remaining: 19.4s
169:	learn: 60.8924196	total: 3.96s	remaining: 19.3s
170:	learn: 60.8314219	total: 3.98s	remaining: 19.3s
171:	learn: 60.8059588	total: 4.01s	remaining: 19.3s
172:	learn: 60.7296755	total: 4.03s	remaining: 19.3s
173:	learn: 60.6962898	total: 4.06s	remaining: 19.3s
174:	learn: 60.6156556	total: 4.08s	remaining: 19.3s
175:	learn: 60.5627256	total: 4.11s	remaining: 19.2s
176:	learn: 60.4722646	total: 4.13s	remaining: 19.2s
177:	learn: 60.4489375	total: 4.16s	remaining: 19.2s
178:	learn: 60.4030824	total: 4.18s	remaining: 19.2s
179:	learn: 60.3300041	total: 4.21s	remaining: 19.2s
180:	learn: 60.3171075	total: 4.23s	remaining: 19.2s
181:	learn: 60.2685394	total: 4.26s	remaining: 19.1s
182:	learn: 60.2553373	total: 4.28s	remaining: 19.1s
183:	learn: 60.2349663	total: 4.3s	remaining: 

327:	learn: 55.1526826	total: 7.99s	remaining: 16.4s
328:	learn: 55.0978648	total: 8.02s	remaining: 16.4s
329:	learn: 55.0690083	total: 8.04s	remaining: 16.3s
330:	learn: 55.0343911	total: 8.07s	remaining: 16.3s
331:	learn: 55.0006309	total: 8.1s	remaining: 16.3s
332:	learn: 54.9848199	total: 8.12s	remaining: 16.3s
333:	learn: 54.9606782	total: 8.14s	remaining: 16.2s
334:	learn: 54.8831340	total: 8.17s	remaining: 16.2s
335:	learn: 54.8473654	total: 8.19s	remaining: 16.2s
336:	learn: 54.8091922	total: 8.21s	remaining: 16.2s
337:	learn: 54.8010544	total: 8.24s	remaining: 16.1s
338:	learn: 54.7828885	total: 8.27s	remaining: 16.1s
339:	learn: 54.7561418	total: 8.29s	remaining: 16.1s
340:	learn: 54.7437844	total: 8.31s	remaining: 16.1s
341:	learn: 54.7299173	total: 8.34s	remaining: 16s
342:	learn: 54.7133540	total: 8.37s	remaining: 16s
343:	learn: 54.6984760	total: 8.39s	remaining: 16s
344:	learn: 54.6849675	total: 8.42s	remaining: 16s
345:	learn: 54.6708144	total: 8.44s	remaining: 16s
346:

488:	learn: 50.7573858	total: 12.2s	remaining: 12.8s
489:	learn: 50.7259924	total: 12.3s	remaining: 12.8s
490:	learn: 50.7084906	total: 12.3s	remaining: 12.7s
491:	learn: 50.6907052	total: 12.3s	remaining: 12.7s
492:	learn: 50.6824249	total: 12.4s	remaining: 12.7s
493:	learn: 50.6331198	total: 12.4s	remaining: 12.7s
494:	learn: 50.6186977	total: 12.4s	remaining: 12.7s
495:	learn: 50.5353969	total: 12.4s	remaining: 12.6s
496:	learn: 50.4981726	total: 12.5s	remaining: 12.6s
497:	learn: 50.4829236	total: 12.5s	remaining: 12.6s
498:	learn: 50.4416543	total: 12.5s	remaining: 12.6s
499:	learn: 50.4219777	total: 12.5s	remaining: 12.5s
500:	learn: 50.4137030	total: 12.6s	remaining: 12.5s
501:	learn: 50.3908883	total: 12.6s	remaining: 12.5s
502:	learn: 50.3661100	total: 12.6s	remaining: 12.5s
503:	learn: 50.3298051	total: 12.7s	remaining: 12.5s
504:	learn: 50.3084513	total: 12.7s	remaining: 12.4s
505:	learn: 50.3006213	total: 12.7s	remaining: 12.4s
506:	learn: 50.2743041	total: 12.7s	remaining:

...

907:	learn: 56.7971385	total: 34.1s	remaining: 3.46s
908:	learn: 56.7946851	total: 34.2s	remaining: 3.42s
909:	learn: 56.7864630	total: 34.2s	remaining: 3.39s
910:	learn: 56.7816101	total: 34.3s	remaining: 3.35s
911:	learn: 56.7763187	total: 34.3s	remaining: 3.31s
912:	learn: 56.7620855	total: 34.3s	remaining: 3.27s
913:	learn: 56.7609985	total: 34.4s	remaining: 3.24s
914:	learn: 56.7493359	total: 34.4s	remaining: 3.2s
915:	learn: 56.7409450	total: 34.5s	remaining: 3.16s
916:	learn: 56.7236098	total: 34.5s	remaining: 3.12s
917:	learn: 56.7189479	total: 34.6s	remaining: 3.09s
918:	learn: 56.6995776	total: 34.6s	remaining: 3.05s
919:	learn: 56.6822989	total: 34.6s	remaining: 3.01s
920:	learn: 56.6747001	total: 34.7s	remaining: 2.98s
921:	learn: 56.6714770	total: 34.7s	remaining: 2.94s
922:	learn: 56.6429041	total: 34.8s	remaining: 2.9s
923:	learn: 56.6401395	total: 34.8s	remaining: 2.86s
924:	learn: 56.6337672	total: 34.9s	remaining: 2.83s
925:	learn: 56.6141158	total: 34.9s	remaining: 2

66:	learn: 71.2446915	total: 3.98s	remaining: 55.4s
67:	learn: 71.1643128	total: 4.02s	remaining: 55.1s
68:	learn: 71.0698666	total: 4.06s	remaining: 54.8s
69:	learn: 70.9931667	total: 4.1s	remaining: 54.5s
70:	learn: 70.9819601	total: 4.14s	remaining: 54.2s
71:	learn: 70.9133905	total: 4.2s	remaining: 54.1s
72:	learn: 70.8752113	total: 4.28s	remaining: 54.3s
73:	learn: 70.7510594	total: 4.36s	remaining: 54.5s
74:	learn: 70.6732720	total: 4.41s	remaining: 54.4s
75:	learn: 70.6298486	total: 4.45s	remaining: 54.1s
76:	learn: 70.5714029	total: 4.5s	remaining: 54s
77:	learn: 70.5583091	total: 4.54s	remaining: 53.7s
78:	learn: 70.5040832	total: 4.58s	remaining: 53.4s
79:	learn: 70.4880392	total: 4.61s	remaining: 53.1s
80:	learn: 70.4368403	total: 4.67s	remaining: 53s
81:	learn: 70.3887087	total: 4.71s	remaining: 52.8s
82:	learn: 70.3461300	total: 4.75s	remaining: 52.5s
83:	learn: 70.3103368	total: 4.79s	remaining: 52.3s
84:	learn: 70.2864868	total: 4.84s	remaining: 52.1s
85:	learn: 70.25885

223:	learn: 65.8690535	total: 10.4s	remaining: 36.1s
224:	learn: 65.8293653	total: 10.5s	remaining: 36s
225:	learn: 65.8275335	total: 10.5s	remaining: 35.9s
226:	learn: 65.7972929	total: 10.5s	remaining: 35.9s
227:	learn: 65.7572394	total: 10.6s	remaining: 35.8s
228:	learn: 65.7506938	total: 10.6s	remaining: 35.7s
229:	learn: 65.7493661	total: 10.7s	remaining: 35.7s
230:	learn: 65.7293224	total: 10.7s	remaining: 35.6s
231:	learn: 65.7169800	total: 10.7s	remaining: 35.5s
232:	learn: 65.7122377	total: 10.8s	remaining: 35.4s
233:	learn: 65.6921603	total: 10.8s	remaining: 35.3s
234:	learn: 65.6685335	total: 10.8s	remaining: 35.2s
235:	learn: 65.6382572	total: 10.9s	remaining: 35.2s
236:	learn: 65.6350345	total: 10.9s	remaining: 35.1s
237:	learn: 65.6056935	total: 10.9s	remaining: 35s
238:	learn: 65.5992881	total: 11s	remaining: 35s
239:	learn: 65.5790231	total: 11s	remaining: 34.9s
240:	learn: 65.5782068	total: 11.1s	remaining: 34.9s
241:	learn: 65.5721421	total: 11.1s	remaining: 34.8s
242

384:	learn: 62.5567865	total: 16.9s	remaining: 27s
385:	learn: 62.5346330	total: 17s	remaining: 27s
386:	learn: 62.5202734	total: 17s	remaining: 26.9s
387:	learn: 62.4979833	total: 17.1s	remaining: 26.9s
388:	learn: 62.4561863	total: 17.1s	remaining: 26.9s
389:	learn: 62.4171386	total: 17.1s	remaining: 26.8s
390:	learn: 62.4079725	total: 17.2s	remaining: 26.8s
391:	learn: 62.3881044	total: 17.2s	remaining: 26.7s
392:	learn: 62.3861737	total: 17.3s	remaining: 26.7s
393:	learn: 62.3748793	total: 17.3s	remaining: 26.6s
394:	learn: 62.3658621	total: 17.4s	remaining: 26.6s
395:	learn: 62.3438858	total: 17.4s	remaining: 26.5s
396:	learn: 62.3073532	total: 17.4s	remaining: 26.5s
397:	learn: 62.2389489	total: 17.5s	remaining: 26.4s
398:	learn: 62.1983957	total: 17.5s	remaining: 26.4s
399:	learn: 62.1781150	total: 17.5s	remaining: 26.3s
400:	learn: 62.1639598	total: 17.6s	remaining: 26.3s
401:	learn: 62.1599324	total: 17.6s	remaining: 26.2s
402:	learn: 62.1418930	total: 17.7s	remaining: 26.2s
4

543:	learn: 59.8141300	total: 23.3s	remaining: 19.6s
544:	learn: 59.8074863	total: 23.4s	remaining: 19.5s
545:	learn: 59.7994510	total: 23.4s	remaining: 19.5s
546:	learn: 59.7826840	total: 23.5s	remaining: 19.4s
547:	learn: 59.7766936	total: 23.5s	remaining: 19.4s
548:	learn: 59.7654790	total: 23.6s	remaining: 19.4s
549:	learn: 59.7487021	total: 23.6s	remaining: 19.3s
550:	learn: 59.7410097	total: 23.6s	remaining: 19.3s
551:	learn: 59.7319904	total: 23.7s	remaining: 19.2s
552:	learn: 59.7093120	total: 23.7s	remaining: 19.2s
553:	learn: 59.7019841	total: 23.7s	remaining: 19.1s
554:	learn: 59.6929151	total: 23.8s	remaining: 19.1s
555:	learn: 59.6844044	total: 23.8s	remaining: 19s
556:	learn: 59.6807457	total: 23.9s	remaining: 19s
557:	learn: 59.6722136	total: 23.9s	remaining: 18.9s
558:	learn: 59.6691299	total: 23.9s	remaining: 18.9s
559:	learn: 59.6177271	total: 24s	remaining: 18.8s
560:	learn: 59.5812151	total: 24s	remaining: 18.8s
561:	learn: 59.5633545	total: 24s	remaining: 18.7s
562